In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from imblearn.over_sampling import RandomOverSampler

In [ ]:
path = './data/descriptor_binary.csv'
df = pd.read_csv(path)
x = df.drop(['shell_number'], axis=1)
y = df['shell_number']


In [ ]:
random_state = 11
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=random_state)
ros = RandomOverSampler(random_state=random_state)
x_resampled, y_resampled = ros.fit_resample(x_train, y_train)


In [ ]:
gbdt = GradientBoostingClassifier(random_state=random_state)

In [ ]:
param_grid = {
    'n_estimators': np.linspace(10,100,5),
    'learning_rate': np.linspace(0.1,0.2,5),
    'max_features': np.linspace(0.1, 1, 5),
    'min_samples_leaf': np.linspace(10,100,5),
    'min_samples_split': np.linspace(3,30,5),
    'max_depth': np.linspace(3,5),
    'subsample': np.linspace(0.5,1,5),
}

grid_search = GridSearchCV(
    estimator=gbdt,
    param_grid=param_grid,
    scoring='accuracy',
    cv=10
)

grid_search.fit(x,y)


In [ ]:
from scipy.stats import uniform, randint
from sklearn.model_selection import RandomizedSearchCV
param_dist = {
    'n_estimators': randint(50, 150),
    'learning_rate': uniform(0.1, 0.2),
    'max_features': uniform(0.1, 1),
    'min_samples_leaf': randint(10, 50),
    'min_samples_split': randint(2, 20),
    'max_depth': randint(3, 15),
    'subsample': uniform(0.5, 1.0)
}
random_search = RandomizedSearchCV(
    estimator=gbdt,
    param_distributions=param_dist,
    n_iter=100,
    cv=5,
    scoring='accuracy',
    random_state=random_state)

random_search.fit(x, y)

best_params = random_search.best_params_
print("最优参数组合:", best_params)
best_accuracy = random_search.best_score_

best_gbdt = gbdt.set_params(**best_params)
cv_results = cross_validate(best_gbdt, x, y, cv=skf, scoring=scoring)
print(cv_results['test_accuracy'].mean())